<a href="https://colab.research.google.com/github/hdgomez18/challenge_2024/blob/main/etl_withPyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **instalacion libreria**

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=948a93ab4af811933f1276e02ce5b10d7b045b989a9cba7d5695218db3fea0fd
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


# Crear session de spark

In [5]:
from pyspark.sql import SparkSession
from pyspark import SparkContext


spSession = SparkSession \
            .builder \
            .appName("pyspark_code") \
            .getOrCreate()

# **Carga de datos**

In [27]:
df_depa = spSession.read.csv("departments.csv", header = 'false',sep= ';')
df_hire = spSession.read.csv("hired_employees.csv", header = 'false',sep= ';')
df_jobs = spSession.read.csv("jobs.csv", header = 'false',sep= ';')

In [42]:
from os import truncate
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,DoubleType,LongType
from pyspark.sql.functions import *
#delete nulls
df_depa = df_depa.where(col('_c0').isNotNull())
#replace columns names
df_depa = df_depa.withColumnRenamed('_c0','id')
df_depa = df_depa.withColumnRenamed('_c1','department')
#cast columns to integer
df_depa = df_depa.withColumn('id',col('id').cast(IntegerType()))
df_depa.show(truncate= False)
#replace columns names
df_hire = df_hire.withColumnRenamed('_c0','id')
df_hire = df_hire.withColumnRenamed('_c1','name')
df_hire = df_hire.withColumnRenamed('_c2','datetime')
df_hire = df_hire.withColumnRenamed('_c3','department_id')
df_hire = df_hire.withColumnRenamed('_c4','job_id')
#separate the rows with nulls
df_without_name = df_hire.where(col('name').isNull() | col('datetime').isNull() |  col('department_id').isNull() |  col('job_id').isNull())

df_hire_f = df_hire.where(col('name').isNotNull() | col('datetime').isNotNull() |  col('department_id').isNotNull() |  col('job_id').isNotNull())
#cast columns to integer
df_hire_f = df_hire_f.withColumn('id',col('id').cast(IntegerType()))
df_hire_f = df_hire_f.withColumn('department_id',col('department_id').cast(IntegerType()))
df_hire_f = df_hire_f.withColumn('job_id',col('job_id').cast(IntegerType()))

df_jobs.show(truncate= False)
df_f2.count()

#replace columns names
df_jobs = df_jobs.withColumnRenamed('_c0','id')
df_jobs = df_jobs.withColumnRenamed('_c1','job')
#cast columns to integer
df_jobs = df_jobs.withColumn('id',col('id').cast(IntegerType()))

CREATE DB AND TABLES

In [62]:
db_name = "COMPANY_AAA"
spSession.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")

DataFrame[]

In [59]:
from os.path import abspath
from pyspark.sql import SparkSession

# warehouse_location points to the default location for managed databases and tables
warehouse_location = abspath('spark-warehouse')

# Create spark session with hive enabled
spark = SparkSession \
    .builder \
    .appName("SparkByExamples.com") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

In [65]:
#CREATE TABLE JOBS
df_jobs.write.mode('overwrite') \
    .saveAsTable("company_aaa.jobs")
#CREATE TABLE DEPARMENTS
df_depa.write.mode('overwrite') \
    .saveAsTable("company_aaa.deparments")
#CREATE TABLE hired_employees
df_hire_f.write.mode('overwrite') \
    .saveAsTable("company_aaa.hired_employees")
#CREATE TABLE hired_employees_withNulls
df_without_name.write.mode('overwrite') \
    .saveAsTable("company_aaa.employeesNulls")


save backup to parquet

In [89]:
df_jobs.write.format("parquet").save("/content/backup/jobs.parquet")
df_depa.write.format("parquet").save("/content/backup/deparment.parquet")
df_hire_f.write.format("parquet").save("/content/backup/employees.parquet")
df_without_name.write.format("parquet").save("/content/backup/employees_null.parquet")

create function to restore table from backup

In [102]:
def restore_table (name_table: str):
    df_backup = spark.read.parquet('/content/backup/'+name_table+'.parquet')
    return print('Please use the next dataframe: df_backup to consult the data of '+name_table)

restore_table('deparment')

Please use the next dataframe: df_backup to consult the data of deparment
